In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
import re

import geopandas as gpd
import requests,json


from shapely import geos
from shapely.geometry import Point
import fiona
from fiona.crs import from_epsg,from_string

from geopandas import GeoDataFrame
from shapely.geometry import Polygon
from geopandas import GeoSeries


from shapely import geometry
from shapely.geometry import shape,LineString,Point,Polygon

import osmnx as ox
import networkx as nx

import folium

from folium import IFrame

In [3]:
def rest(address):
    try:
        url = 'https://api.tomtom.com/search/2/geocode/{}.json?&key=h2n93GznJsnsDHtv2vPpdT9GmJijp9o9&countrySet=TWN&language=zh-TW&limit=1'.format(address)
        response = requests.get(url)
        data = response.text
        js = json.loads(str(data))
        return js['results'][0]['position']
    except Exception as e:
        print(f'An Error occurred: {e}')
        return {'lat':0,'lon':0}
    
def trans(gdf):
    gdf.crs = {'init' :'epsg:3824'} # 避免資料沒設，這邊再重新給一次
    gdf=gdf.to_crs({'init' :'epsg:4326'})
    return gdf

def utf2asc(s):
    return str(str(s).encode('ascii', 'xmlcharrefreplace'))[2:-1]

heading3 = """<h3>{}</h3>""".format

In [5]:
import warnings
warnings.filterwarnings("ignore")

# 設定路徑

In [6]:
import os
os.chdir(r'C:\Users\RT009223\TestriteGroup\滲透率專案\02_adjusted data')

# TLW

In [9]:
df_04=pd.read_excel(r'C:\Users\RT009223\TestriteGroup\滲透率專案\01_raw data\TLW2019一般卡會員(無完整地址).xlsx')
print(df_04.shape)
df_04['addr']=df_04.addr.apply(lambda x:str(x))

(212650, 3)


In [10]:
df_04_1=df_04[df_04.addr.apply(lambda x:len(x)>9)].reset_index(drop=True)
df_04_1.shape

(140827, 3)

In [11]:
pattern = re.compile(r'..[市,縣].+[路,街].+')
location=df_04_1.addr.apply(lambda x:pattern.findall((x)))

addr01=[]
for i in location:
    if len(i)==1:
        addr01.append(i[0])      
    else:
        addr01.append(np.nan)

In [12]:
df_04_1.addr=addr01
df_04_1=df_04_1.dropna(subset=['addr']).reset_index(drop=True)
df_04_1.shape

(111380, 3)

In [13]:
df_04_1['lon']=float(0)
df_04_1['lat']=float(0)
df_04_1['check']=0
df_04_1.head()

,id,addr,zone,lon,lat,check
0,20121112000470,台北市中正區莊靜路478號4樓之8,100.0,0.0,0.0,0
1,20111026000233,台南市南 區中華南路二段400巷8號4樓,702.0,0.0,0.0,0
2,20190807000649,彰化縣和美鎮柑井里柑竹路21巷14號,508.0,0.0,0.0,0
3,20050408000436,台北市信義區臥龍街427巷65號,110.0,0.0,0.0,0
4,20081210000229,台中市大里區豐中三街48號,412.0,0.0,0.0,0


In [14]:
dt_TLW=df_04_1.copy()

# HOLA

In [15]:
df_04=pd.read_excel(r'C:\Users\RT009223\TestriteGroup\滲透率專案\01_raw data\HOLA2019一般卡會員(無完整地址).xlsx')
print(df_04.shape)
df_04['addr']=df_04.addr.apply(lambda x:str(x))

(117589, 3)


In [16]:
df_04_1=df_04[df_04.addr.apply(lambda x:len(x)>9)].reset_index(drop=True)
df_04_1.shape

pattern = re.compile(r'..[市,縣].+[路,街].+')
location=df_04_1.addr.apply(lambda x:pattern.findall((x)))

addr01=[]
for i in location:
    if len(i)==1:
        addr01.append(i[0])      
    else:
        addr01.append(np.nan)

In [17]:
df_04_1.addr=addr01
df_04_1=df_04_1.dropna(subset=['addr']).reset_index(drop=True)
df_04_1.shape

(53161, 3)

In [18]:
df_04_1['lon']=float(0)
df_04_1['lat']=float(0)
df_04_1['check']=0
df_04_1.head()

,id,addr,zone,lon,lat,check
0,20050204000250,高雄市鼓山區九如４路１９９５號,804.0,0.0,0.0,0
1,19990101001192,新北市中和區興街6巷13號11樓,235.0,0.0,0.0,0
2,19970302000405,桃園市中壢區長樂4街10-3號12樓,320.0,0.0,0.0,0
3,20070421000110,新北市新莊區中山路３段６０８號３樓,242.0,0.0,0.0,0
4,20040612000852,高雄市左營區楠梓區秀群路457巷17號11樓,813.0,0.0,0.0,0


In [19]:
dt_HOLA=df_04_1.copy()

# 現有 get back

In [7]:
df_01=pd.read_excel(r'get back.xlsx')
df_01.shape

(133014, 6)

In [21]:
print(dt_TLW.shape,dt_TLW.id.isin(df_01.id).sum())
print(dt_HOLA.shape,dt_HOLA.id.isin(df_01.id).sum())

(111380, 6) 111380
(53161, 6) 31527


In [23]:
dt_TLW_02=dt_TLW[-dt_TLW.id.isin(df_01.id)]
dt_HOLA_02=dt_HOLA[-dt_HOLA.id.isin(df_01.id)]

In [27]:
df_01=pd.concat([df_01,dt_TLW_02,dt_HOLA_02]).reset_index(drop=True)

In [48]:
df_01.shape

(133014, 6)

## search 經緯度

In [27]:
def rest(address):
    try:
        url = 'https://api.tomtom.com/search/2/geocode/{}.json?&key=t3yvR26O7tar1UvnAlOaAzCtqxGA1XRA&countrySet=TWN&language=zh-TW&limit=1'.format(address)
        response = requests.get(url)
        data = response.text
        js = json.loads(str(data))
        return js['results'][0]['position']
    except Exception as e:
        print(f'An Error occurred: {e}')
        return {'lat':0,'lon':0}

In [28]:
rest('花蓮縣吉安鄉東海15街42-3號')

{'lat': 23.96755, 'lon': 121.59561}

In [29]:
for i in range(45400,47800):
    print(i,df_01['addr'][i])
    pos=rest(df_01['addr'][i])
    df_01['lat'][i]=float(pos['lat'])
    df_01['lon'][i]=float(pos['lon'])
    df_01['check'][i]=1
    print(df_01['lat'][i],df_01['lon'][i])

45400 台南市善化區小新里陽光二路151巷22號
23.11875 120.30621
45401 桃園市蘆竹區南崁中正路25號23樓之1
25.05239 121.28433
45402 台中市南屯區南區文心南路578號12F之1
24.13253 120.63867
45403 雲林縣水林鄉110雲林縣水林鄉蔦松路155-6號
23.51542 120.23045
45404 花蓮縣花蓮市民德４街３５號
23.98534 121.6226
45405 新北市汐止區民族街５巷１４號５Ｆ
25.0703 121.6311
45406 台中市北　區陝西東1街30號
24.16972 120.66911
45407 台南市東　區崇明２街１２號
22.98066 120.21544
45408 桃園市楊梅區xx路201巷75弄45號
24.95549 121.17199
45409 花蓮縣新城鄉北埔村自強路３２號
24.03314 121.60416
45410 桃園市楊梅區幼獅工業區獅三路10號DM退990113
24.93284 121.15818
45411 台南市仁德區長興2 街68號
22.99186 120.25787
45412 台南市中西區中區中華南路二段３１５巷３弄２９－４五樓
22.98764 120.18529
45413 新北市中和區環北路358號19樓之3
24.99297 121.50202
45414 桃園市平鎮區華勛街37巷6弄20號
24.95136 121.25844
45415 花蓮縣花蓮市國富２８街２８號
23.98819 121.59901
45416 新北市中和區警安路132巷7號5樓
24.99424 121.50469
45417 台中市北　區陜西2街21-13號7樓
25.05982 121.51018
45418 台南市中西區中區南門路75號9F
22.98944 120.2049
45419 彰化縣彰化市寶?路121巷8號
An Error occurred: 'results'
0.0 0.0
45420 桃園市八德區八德自強街４６號
24.96801 121.30241
45421 桃園市平鎮區市德育路15號
24.95312 121.22029
45422 台南市北　區育德２路２００巷２號
23.01197 

22.65708 120.30694
45586 台中市北屯區軍福19路199號
24.1697 120.72255
45587 高雄市大寮區六合路2-26號5樓
22.58978 120.37082
45588 台中市北屯區軍榮３街８９號
24.17923 120.71393
45589 台中市南屯區永春東2路800號14樓之3
24.13189 120.63735
45590 台中市大里區萬板路46巷19號4樓
24.10504 120.70334
45591 桃園市八德區吉昌街94號6樓
24.95407 121.29515
45592 台北市中山區忠孝東路一段112號9樓
25.04716 121.52837
45593 花蓮縣花蓮市國盛里光復街97之18號5樓
23.97974 121.60523
45594 台南市中西區中區健康路１段３９６號１３樓之６
22.98131 120.20493
45595 台中市北屯區安順東8街47號
24.17564 120.67841
45596 花蓮縣花蓮市國富２２街１６號
23.99918 121.59977
45597 台南市新營區富強路６１號２Ｆ１
23.06813 120.28647
45598 台南市北　區公學路五段697巷155號九樓
23.07148 120.15903
45599 台南市永康區台南永華路二段819號19F-3
22.99105 120.16856
45600 高雄市三民區津街２２１巷１３號
22.64818 120.3
45601 台中市北屯區軍福18路311號8樓之2
24.17796 120.7189
45602 高雄市新興區新莊一路393號9樓之3
22.6268 120.30347
45603 台中市南屯區大墩14街328號18樓之5
24.13727 120.65235
45604 台中市西屯區朝馬６街６０－７號
24.16912 120.63553
45605 台北市松山區明權東路6段90巷6弄3號6樓
25.06714 121.58625
45606 桃園市中壢區九和3街15號5樓
24.96521 121.22427
45607 桃園市桃園區桃園市南興街21號
24.99046 121.30572
45608 台南市新市區鄉和平街１３８巷３６號
23.08068 120.

25.05551 121.5134
45775 台中市北屯區安順五街23號
24.17517 120.68261
45776 台北市信義區忠孝路５段３９０號２ｆ
25.04102 121.57253
45777 台南市永康區復華１０街１７號
23.01229 120.24658
45778 基隆市安樂區五路12號1
25.13173 121.73422
45779 桃園市中壢區中壢區新中北路374號
24.95873 121.24278
45780 台中市西　區士林區後街２１巷４６弄１號５樓　　　　　　　　　　　　
An Error occurred: list index out of range
0.0 0.0
45781 台南市歸仁區大順５街２１巷１２號
22.96485 120.28704
45782 桃園市中壢區金峰五街53號7樓
24.94623 121.24951
45783 雲林縣斗六市武凌街４９巷８號３樓
23.69923 120.53548
45784 桃園市楊梅區高榮里新榮路223巷217號
24.9559 121.1616
45785 桃園市中壢區中壢延平路645號9樓之2
24.96639 121.23448
45786 台中市西屯區工業36路19號
24.18128 120.61452
45787 新北市中和區秀郎路3段60巷24號
24.9951 121.52549
45788 台北市中正區基隆路２段１９６巷１１樓３３
25.02927 121.55964
45789 桃園市中壢區桃園市中壢區仁慈路350巷162號
24.94495 121.2525
45790 台中市南屯區大墩13街7號7樓
24.1377 120.65405
45791 高雄市三民區重上街153號9樓
22.64614 120.30212
45792 桃園市龜山區光峰路212-1號8F
25.00206 121.34241
45793 桃園市龜山區文三三街38巷16號6F
25.05595 121.35883
45794 雲林縣斗六市斗六五福街７０巷２３號
23.70663 120.55419
45795 桃園市八德區長興街498號
24.93256 121.27295
45796 桃園市新屋區頭洲里福州路53'巷1號
24.96305 121.14399
4579

22.68664 120.48975
45962 高雄市小港區孔宅8路28號
22.57884 120.37414
45963 台北市中山區敦化南路一段7號10樓之一
25.04564 121.54776
45964 台中市南　區南  區學府路135巷2弄11號3樓之2
24.12871 120.67367
45965 台南市永康區復華６街１８號
23.01227 120.24582
45966 台北市士林區大大南路４６號
25.08877 121.52143
45967 桃園市桃園區德惠街13號
25.0182 121.29093
45968 台中市南　區德?街15巷9號5樓之3
An Error occurred: 'results'
0.0 0.0
45969 台南市東　區崇德2街76巷31號
22.98085 120.22178
45970 桃園市中壢區中壢長沙路106號
24.94943 121.23134
45971 台南市中西區西區樹林街２段１０９號５樓
22.9859 120.20425
45972 台北市南港區台北市南港區福德街469號15樓
25.03903 121.59298
45973 台南市中西區西區忠明街5號
22.99588 120.19713
45974 台北市北投區新明路５１－６號４樓之１
25.05504 121.42985
45975 台中市中　區黎明路二段４２３號１３樓－１
24.14591 120.63622
45976 台北市士林區泰山路９５巷２－５號
25.11884 121.54251
45977 高雄市左營區新土街77號DM退件0802
22.66656 120.31076
45978 台中市北　區潭子鄉頭張路2段71號10F
24.19408 120.69618
45979 台中市潭子區勝利12街87巷41號12樓
24.21304 120.69981
45980 高雄市楠梓區大學28街230巷10號
22.73026 120.29171
45981 台南市仁德區南寧街14號
22.95264 120.24999
45982 台中市西屯區台中港路2段184巷26號10樓-1
24.1692 120.65041
45983 花蓮縣花蓮市國興1街133巷3號
23.99649 121.60409
45984 新北市汐止

24.92221 121.2934
46149 花蓮縣花蓮市國民8街106號2-1
23.99472 121.60405
46150 宜蘭縣五結鄉大吉鄉大吉三路302巷3弄7號
24.69537 121.80093
46151 台南市永康區富強3街7號6樓
23.01607 120.27384
46152 台南市中西區中區大同路２段７５０巷１６４弄８號
22.96586 120.21056
46153 台北市士林區通和西街1段130號5樓
25.08743 121.51188
46154 新北市汐止區汐止鎮汐萬路2段275巷1弄21
25.0883 121.64623
46155 高雄市燕巢區芎林村芎招路6之1號8樓
22.78998 120.32185
46156 台北市信義區台北市台北市忠孝東路五段295巷6弄36號二樓
25.04171 121.574
46157 台北市中山區忠孝東路2段88號12樓
25.04353 121.54151
46158 高雄市仁武區仁和村澄和街１１６號
22.70364 120.34764
46159 新北市土城區學勤路６５７號２樓
24.97928 121.44682
46160 桃園市中壢區中壢榮安7街55巷12號
24.96174 121.26026
46161 台中市潭子區勝利６街１９號４樓
24.20969 120.69799
46162 嘉義市嘉義市西門街．．．．．．．．．．．．．．．．
An Error occurred: list index out of range
0.0 0.0
46163 桃園市蘆竹區蘆竹五福一路69-8號6樓
25.05678 121.29484
46164 嘉義縣中埔鄉和美村中華路798巷21號
23.44808 120.45962
46165 台南市永康區鹽行路10號
23.0422 120.23646
46166 花蓮縣吉安鄉南埔16街51巷9號
23.97138 121.58858
46167 台北市內湖區台北巿內湖路一段373巷19
25.07931 121.57967
46168 台北市大同區大同區承德路三段263號11樓之5
25.06544 121.51895
46169 高雄市左營區明成貳路４０６號
22.69143 120.30084
46170 桃園市平鎮區山頂段東

...

22.973 120.24485
46899 高雄市三民區天公街42號3F-2
22.66578 120.32368
46900 台中市南屯區工業區３７路３４號
24.16206 120.60628
46901 台南市中西區中區中和路５９號
23.00558 120.18511
46902 彰化縣芬園鄉宗新街１７０號
24.01395 120.66254
46903 台南市中西區西區成功路327號
22.99981 120.20021
46904 桃園市蘆竹區五福村六福路157號7
25.05474 121.29578
46905 宜蘭縣羅東鎮羅東振興東路159號
24.6808 121.76923
46906 桃園市桃園區民富17街43號6樓
24.92808 121.06993
46907 高雄市鳳山區忠信街31號
22.627 120.34698
46908 台南市安南區南區長溪路三段273巷15號
23.06599 120.19716
46909 宜蘭縣羅東鎮光華路269-13號11樓   (無此號)
24.67658 121.77749
46910 彰化縣彰化市南瑤路311號
24.07279 120.54034
46911 花蓮縣吉安鄉慈惠4街170巷16號
23.97853 121.58736
46912 台中市豐原區豐原區信義街200號3樓之1
24.25443 120.71871
46913 桃園市桃園區新埔7街169巷11號3號　
25.02412 121.29881
46914 桃園市八德區中正北路72號
24.93239 121.28385
46915 台中市西　區中美街84號
24.14402 120.66039
46916 台中市北　區台中市北區美德街179之4號5樓
24.1581 120.68019
46917 台中市豐原區平勢路二段２９０巷１號
24.2628 120.74241
46918 花蓮縣花蓮市華明街８巷１３號
24.00957 121.62206
46919 台北市內湖區行扇路48巷9號
25.05718 121.57552
46920 雲林縣斗六市棒球15街10號
23.72072 120.53934
46921 桃園市中壢區執信２街１號４樓
24.94403 121.22539
46922 台南市安平區建平17街23

24.95969 121.23251
47089 台中市太平區　大興二十一街１８０號　　　　　　　　　　　　
An Error occurred: list index out of range
0.0 0.0
47090 屏東縣三地門三地村中正路2段71巷9號
22.71618 120.65255
47091 台北市松山區員林街164巷58弄3號
25.03815 121.57465
47092 花蓮縣吉安鄉東里１５街４８號
23.96388 121.59145
47093 台南市永康區復國１路２１０巷２７號
23.01224 120.24836
47094 台南市南　區鹽埕路103巷13之1號
22.96651 120.18866
47095 台北市大同區哈蜜街５４號４樓
25.07409 121.51734
47096 桃園市中壢區大壽街523號
24.97122 121.18783
47097 高雄市三民區菜公里鼎泰街231號9樓
22.66637 120.31815
47098 台中市太平區大興11街125巷51號
24.1521 120.7305
47099 高雄市鹽埕區苓雅區大義街103號
22.62176 120.28615
47100 桃園市中壢區榮安１４街８－５號５Ｆ
24.96522 121.26002
47101 台南市安平區育平9街372巷189弄56號
22.99723 120.1656
47102 台南市永康區東橋12街55號10F-5
23.01824 120.22509
47103 彰化縣和美鎮湖內里湖竹路163巷17號
24.14963 120.50333
47104 高雄市左營區州仔路１２７巷１弄８號
22.68587 120.3011
47105 台南市南　區郡安路四段８９巷３３弄９號
23.02711 120.21032
47106 台北市中山區吉臨路199巷23號4樓
25.05999 121.53103
47107 高雄市三民區瑞豐街９－２號３樓
22.64116 120.33146
47108 台北市士林區壺蘆街135巷15弄5號3樓
25.08483 121.50767
47109 桃園市平鎮區市漢口街83巷2弄14號
24.90985 121.20691
47110 花蓮縣花蓮市民德４街3之6號
23.98541 

25.10048 121.52393
47277 桃園市觀音區草漯村２２鄰四維一路５７號４樓之３
An Error occurred: list index out of range
0.0 0.0
47278 台中市北　區自強路41巷63號
24.15381 120.69512
47279 新北市中和區成功南路１２９巷１８－３號
24.9929 121.52612
47280 台南市新市區富林路110號4樓之5
23.0824 120.29503
47281 新北市三重區光明街51號9f
25.06202 121.46634
47282 台中市北　區北屯區山西路二段96號5F-2
24.18386 120.67735
47283 台北市內湖區欣明路２４６巷２９號４樓
25.05707 121.58353
47284 台中市北屯區興安路2段214號9樓
24.17967 120.69511
47285 彰化縣員林鎮三信里中三路二段10樓之2號
23.94066 120.38315
47286 高雄市楠梓區大學52街96號
22.7253 120.28309
47287 台中市后里區泉洲路４之３０巷５號
24.31375 120.72951
47288 台南市永康區復華7街117巷25號
23.01518 120.24686
47289 台北市松山區台北巿復興北路179號8樓7室
25.05117 121.54441
47290 彰化縣員林鎮11111111111111勿路12號
An Error occurred: list index out of range
0.0 0.0
47291 台中市西屯區中港路三段１１８號３樓
24.18831 120.61752
47292 台中市北屯區安順４街３３號７樓之３
24.17441 120.69014
47293 桃園市中壢區內壢大華路93號
24.97912 121.25879
47294 台中市北　區九龍街３２號
24.1657 120.69622
47295 台中市大里區成功路562號
24.08657 120.70168
47296 台南市中西區中區　　法華街１９２號　　　　　　　　　　　　　　　
An Error occurred: list index out of range
0.0 0.0
47297 台

24.89875 121.28509
47466 台南市東　區崇善15街5-20號
22.96954 120.23335
47467 高雄市小港區高坪２８街１５９號
22.5423 120.38279
47468 雲林縣古坑鄉崁腳村南昌路66號
23.61373 120.53984
47469 台南市仁德區文化路１１巷５號
22.9715 120.24065
47470 高雄市苓雅區忠孝路256巷57號5F
22.6173 120.31053
47471 高雄市左營區民族ㄧ路901巷26弄2號10樓之5
22.68139 120.31907
47472 彰化縣彰化市天祥路98巷4號
24.06572 120.53356
47473 高雄市鼓山區明誠里文信街256號7樓
22.66143 120.29374
47474 雲林縣林內鄉九芳村大埔路3-3號
23.74553 120.60758
47475 桃園市平鎮區平鎮鎮市湧安里自由街5巷34號
24.89048 121.2032
47476 桃園市桃園區桃園巿仁一街23號
24.99514 121.30044
47477 高雄市鳳山區過碑路１３７號
22.59318 120.3397
47478 台中市中　區福壽街5-5號
25.04897 121.45864
47479 台中市西屯區大墩18街183號9樓
24.13822 120.64907
47480 高雄市小港區貴陽路257巷3號7F
22.5817 120.35356
47481 台南市北　區木港街110巷22號
23.01456 120.19052
47482 雲林縣虎尾鎮埒內路136-73號
23.72219 120.44524
47483 高雄市鼓山區屏路507號-5樓DM退9800924
22.66265 120.29888
47484 新北市三重區福音街3巷4弄20號2F
25.07735 121.48171
47485 桃園市中壢區內壢環中東路１６７號５樓之３
24.95996 121.25323
47486 台北市文山區文山政大二街１６１巷５６號２樓
24.98878 121.58712
47487 台南市歸仁區大吉２街３２號
22.9638 120.29069
47488 台南市中西區中區法華街１７６號５ｆ－１
22.98208 120.20

24.92433 121.28945
47654 台中市西屯區大墩20街183號6樓之1
24.15724 120.65247
47655 新北市中和區華新街109巷53號9F(地址欠
24.9808 121.50968
47656 新北市中和區永元路17巷7弄8號
25.00044 121.52095
47657 台中市豐原區丰東路２３６號
24.2502 120.73192
47658 台中市北屯區竹路二段362-9號8樓之2
24.18631 120.69887
47659 台中市北　區大連路三段236號
24.17373 120.69635
47660 高雄市鳳山區鳳山區文衡路568號12樓
22.64186 120.34718
47661 新北市中和區成功南路25號7樓
24.9929 121.52612
47662 高雄市苓雅區福得一路２６６號
22.62803 120.31559
47663 基隆市安樂區樂利3街81號3F
25.12628 121.71521
47664 台北市中山區敬業3路162巷43號4樓
25.07915 121.55837
47665 台北市文山區西寧街３６巷１１號１４樓
25.0028 121.56521
47666 台中市北屯區北區崇安街82-4號
24.16465 120.69795
47667 台中市南屯區大墩12街369號13樓之2
24.14429 120.64849
47668 桃園市平鎮區324新光路3段150號
24.92625 121.20589
47669 台中市西屯區中區甘州一街1號
24.16506 120.65648
47670 台中市北屯區部子路２７２號
24.16891 120.73337
47671 彰化縣和美鎮健康新村1街5號
24.11031 120.53089
47672 花蓮縣新城鄉康樂村文化路１號
24.04852 121.60473
47673 台南市安平區建平12街134號2樓之2
22.98496 120.17539
47674 宜蘭縣員山鄉金吉一路1巷40弄35號
24.74318 121.73952
47675 高雄市三民區明街7號DM退981019
22.63773 120.31295
47676 彰化縣鹿港鎮路草路三段７０巷４２弄２０號
24.08949 120.448

In [35]:
df_01.loc[[47800-1,47800],:]

NameError: name 'df_01' is not defined

In [31]:
df_01.to_excel(r'get back.xlsx',index=0)

In [33]:
reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [36]:
import gc
gc.collect()

48

In [37]:
#常用的：
import psutil
import os


info = psutil.virtual_memory()
print (u'内存使用：',psutil.Process(os.getpid()).memory_info().rss)
print (u'总内存：',info.total)
print (u'内存占比：',info.percent)
print (u'cpu个数：',psutil.cpu_count())

内存使用： 164757504
总内存： 8460259328
内存占比： 58.9
cpu个数： 8
